<!-- <a href="https://colab.research.google.com/github/Robaina/Pynteny/blob/main/docs/examples/example_api_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> -->

<div style="text-align:center;">
<img src="https://user-images.githubusercontent.com/21340147/227912321-f76e622a-684d-48a9-8ead-9a2ce7caebe9.png" style="width:70%;"/>
</div>
<br/>

[Semidán Robaina](https://github.com/Robaina), February 2023.

In this Notebook, we will use MetaTag through its Python API to reconstruct a phylogenetic tree. To this end, we will use peptide sequences from the [MARref database](https://mmp2.sfb.uit.no/marref/) and a profile HMM to identify sequences beloging to the X gene.

- Note that we could have conducted the same search through Pynteny's command-line interface.

- Find more info in the [documentation pages](https://robaina.github.io/MetaTag/)!

Let's start by importing some required modules.

## DATASETS

1) https://www.nature.com/articles/s41586-022-04862-3
2) https://microbiomics.io/ocean/supp_info/
3) https://sunagawalab.ethz.ch/share/microbiomics/ocean/suppl_data/

In [1]:
from pathlib import Path
import pandas as pd
import pyfastx
from metatag.cli import MetaTag
from metatag.utils import DictMerger
from metatag.visualization import make_tree_html
from metatag.pipelines import ReferenceTreeBuilder, QueryLabeller, QueryProcessor

Let's now create a directory to store results

In [2]:
workdir = Path("example_api")
outdir = workdir / "results"
outdir.mkdir(exist_ok=True, parents=True)

## Download MarRef database:

Download the [MarRef](https://mmp2.sfb.uit.no/marref/) database and extract contents. We will use the `protein.faa` file, containing translated peptide sequences.

We alo need to download two profile HMMs, click on them two download: [TIGR04244](https://ftp.ncbi.nlm.nih.gov/hmm/current/hmm_PGAP.HMM/TIGR04244.1.HMM) and [TIGR04246](https://ftp.ncbi.nlm.nih.gov/hmm/current/hmm_PGAP.HMM/TIGR04246.1.HMM).

## Retrieve GTDB taxonomy from MarRef

To this end, we will extract that info from the MAR ref database itself, which provides a metadata file that contains GTDB taxonomical information for each genome in it.

## Infering a gene-specific phylogenetic tree

We will infer a phylogenetic tree for the gene [_nosZ_](https://www.uniprot.org/uniprotkb/Q51705/entry), encoding a nitrous oxide reductase that participates in the nitrogen cycle. To this end, we will use two TIGRFAM profile HMMs: [TIGR04244.1](https://www.ncbi.nlm.nih.gov/genome/annotation_prok/evidence/TIGR04244/), which encondes a TAT-dependent nitrous-oxide reductase, and [TIGR04246.1](https://www.ncbi.nlm.nih.gov/genome/annotation_prok/evidence/TIGR04246/), which encondes a SEC-dependent nitrous-oxide reductase.

The class `ReferenceTreeBuilder` will take care of all necessary steps to infer the tree. Namely, (i) preprocess the input marref database, (ii) build a reference database containing a maximum of 20 nifH and 5 BCHX representative sequences, using both [CD-Hit](https://github.com/weizhongli/cdhit) and [RepSet](https://onlinelibrary.wiley.com/doi/10.1002/prot.25461), (iii) align the reference sequences with [MUSCLE](https://github.com/EddyRivasLab/hmmer), (iv) infer a phylogenetic tree from the alignment with [FastTree](https://github.com/PavelTorgashov/FastTree).

In [3]:
tree_builder = ReferenceTreeBuilder(
    input_database=Path("/home/robaina/Databases/OceanMicrobiome/ocean_microbiome.faa"),
    hmms=[
        workdir / "data" / "TIGR04244.1.HMM",
        workdir / "data" / "TIGR04246.1.HMM",
    ],
    maximum_hmm_reference_sizes=[100, 100],
    relabel_prefixes=["ref44_", "ref46_"],

    relabel=True,
    remove_duplicates=True,
    hmmsearch_args="--cut_ga",
    output_directory=outdir,
    msa_method="muscle",
    tree_method="fasttree",
    tree_model="JTT",
)
tree_builder.run()

2023-04-08 22:59:58,834 | INFO: Removing duplicates...
2023-04-08 23:02:01,402 | INFO: Asserting correct sequence format...
2023-04-08 23:08:59,960 | INFO: Done!
2023-04-08 23:08:59,961 | INFO: Making peptide-specific reference database...
2023-04-08 23:08:59,962 | INFO: Processing hmm TIGR04244.1 with additional arguments: --cut_ga
2023-04-08 23:08:59,963 | INFO: Running Hmmer...
2023-04-08 23:09:32,214 | INFO: Parsing Hmmer output file...
2023-04-08 23:09:32,268 | INFO: Filtering Fasta...
2023-04-08 23:09:41,495 | INFO: Filtering sequences by established length bounds...
2023-04-08 23:09:41,869 | INFO: Finding representative sequences for reference database...


2023-04-08 23:09:42,155 INFO:Reading PI database...
2023-04-08 23:09:42,155 INFO:Building dataframe
2023-04-08 23:09:42,177 INFO:Dataframe built


2023-04-08 23:09:42,684 | INFO: Relabelling records in reference database...
2023-04-08 23:09:42,686 | INFO: Processing hmm TIGR04246.1 with additional arguments: --cut_ga
2023-04-08 23:09:42,686 | INFO: Running Hmmer...


2023-04-08 23:09:42,550 INFO:Finished building database...
2023-04-08 23:09:42,550 INFO:Starting mixture of summaxacross and sumsumwithin with weight 0.5...
2023-04-08 23:09:42,550 INFO:Repset size: 100


2023-04-08 23:10:15,324 | INFO: Parsing Hmmer output file...
2023-04-08 23:10:15,339 | INFO: Filtering Fasta...
2023-04-08 23:10:25,570 | INFO: Filtering sequences by established length bounds...
2023-04-08 23:10:25,937 | INFO: Finding representative sequences for reference database...


2023-04-08 23:10:26,224 INFO:Reading PI database...
2023-04-08 23:10:26,224 INFO:Building dataframe
2023-04-08 23:10:26,242 INFO:Dataframe built


2023-04-08 23:10:26,670 | INFO: Relabelling records in reference database...
2023-04-08 23:10:26,675 | INFO: Done!
2023-04-08 23:10:26,676 | INFO: Aligning reference database...


2023-04-08 23:10:26,538 INFO:Finished building database...
2023-04-08 23:10:26,538 INFO:Starting mixture of summaxacross and sumsumwithin with weight 0.5...
2023-04-08 23:10:26,538 INFO:Repset size: 100


2023-04-08 23:10:28,842 | INFO: Inferring reference tree...
2023-04-08 23:10:43,829 | INFO: Done!
2023-04-08 23:10:43,831 | INFO: Relabelling tree...
2023-04-08 23:10:44,121 | INFO: Done!


To visualize the generated tree, we will employ [empress](https://github.com/biocore/empress), which generates a web-based interactive tree. The following function calls empress and generates the html file. Click on the image to open the interactive tree in the browser.

In [4]:
make_tree_html(tree_builder.reference_tree, output_dir=outdir / "tree_plot")

/home/robaina/miniconda3/envs/metatag-dev/lib/python3.10/site-packages/empress/tree.py:79: TreeFormatWarning: Internal node names in the tree are not unique.
  warnings.warn(


<a href="file:///home/robaina/Documents/MetaTag/docs/examples/example_api/tree_plot/empress.html" target="_blank"><img src="example_api/example_tree.png" style="width:50%;"></a>

## Preprocess metagenomic data

We need to first preprocess the metagenomic data to remove low quality reads as well as to prefilter sequences using the same profile HMM used to infer the phylogenetic tree. This will reduce the computational cost of the placement step. To this end, we can use the `QueryPreprocessor` class, which contains all necessary steps to preprocess the metagenomic data.

In [5]:
processor = QueryProcessor(
    input_query=Path("/home/robaina/Databases/Uniprot/uniprot_sprot.fasta"),
    hmms=[workdir / "data" / "TIGR04244.1.HMM"],
    hmmsearch_args="--cut_ga",
    minimum_sequence_length=30,
    output_directory=outdir,
)
processor.run()

2023-04-08 23:14:56,134 | INFO: Removing duplicates...
2023-04-08 23:14:59,916 | INFO: Asserting correct sequence format...
2023-04-08 23:15:16,493 | INFO: Done!
2023-04-08 23:15:16,494 | INFO: Making peptide-specific reference database...
2023-04-08 23:15:16,495 | INFO: Processing hmm TIGR04244.1 with additional arguments: --cut_ga
2023-04-08 23:15:16,496 | INFO: Running Hmmer...
2023-04-08 23:15:17,595 | INFO: Parsing Hmmer output file...
2023-04-08 23:15:17,597 | INFO: Filtering Fasta...
2023-04-08 23:15:17,801 | INFO: Filtering sequences by established length bounds...
2023-04-08 23:15:17,811 | INFO: No reduction algorithm has been selected.
2023-04-08 23:15:17,836 | WARNING: Could not delete temporary files: [Errno 2] No such file or directory: '/tmp/tmpo0icvlwi'
2023-04-08 23:15:17,838 | INFO: Done!


## Place and label metagenomic data

We are now ready to place our environmental sequences onto the reference tree to infer their taxonomy and function. To this end, we will employ the `QueryLabeller` class, which will take care of all necessary steps to place  and label the metagenomic data. Namely, (i) preprocess the metagenomic data, (ii) place the sequences onto the reference tree with [papara](https://cme.h-its.org/exelixis/web/software/papara/index.html) and [epa-ng](https://github.com/pierrebarbera/epa-ng), and (iii) label placed sequences with [gappa](https://github.com/lczech/gappa).

In [6]:
labeller = QueryLabeller(
    input_query=processor.filtered_query,
    reference_alignment=tree_builder.reference_alignment,
    reference_tree=tree_builder.reference_tree,
    reference_labels=[
        tree_builder.reference_labels
    ],
    tree_model="JTT",
    alignment_method="papara",
    output_directory=outdir,
    maximum_placement_distance=1.0,
    distance_measure="pendant_diameter_ratio",
    minimum_placement_lwr=0.8,
)
labeller.run()

2023-04-08 23:16:27,592 | INFO: Removing duplicates...
2023-04-08 23:16:27,621 | INFO: Asserting correct sequence format...
2023-04-08 23:16:27,624 | INFO: Data already translated!
2023-04-08 23:16:27,625 | INFO: Relabelling records...
2023-04-08 23:16:27,628 | INFO: Done!
2023-04-08 23:16:27,629 | INFO: Placing reads on tree...
2023-04-08 23:16:33,313 | INFO: Writing tree with placements...
2023-04-08 23:16:33,328 | INFO: Done!
2023-04-08 23:16:33,330 | INFO: Filtering placements by maximum distance: "pendant_diameter_ratio" of 1.0
2023-04-08 23:16:33,440 | INFO: Filtering placements for tree diameter: 3.4100473290000006
2023-04-08 23:16:33,442 | INFO: Filtering placements by minimum LWR of: 0.8
2023-04-08 23:16:33,896 | INFO: Done!
2023-04-08 23:16:33,898 | INFO: Counting labelled placements...
2023-04-08 23:16:34,801 | INFO: Done!
2023-04-08 23:16:34,804 | INFO: Relabelling tree...
2023-04-08 23:16:35,200 | INFO: Done!


## Display placements on the reference tree

We can now visualize the placements of the metagenomic data onto the reference tree. The tree was generated by the `QueryLabeller` using [gappa graft](https://github.com/lczech/gappa/wiki/Subcommand:-graft).

In [8]:
labels = DictMerger.from_pickle_paths(
    [tree_builder.reference_labels, labeller.query_labels]
)

make_tree_html(
    labeller.placements_tree, output_dir=outdir / "tree_plot_placements",
    feature_metadata=labels
    )

/bin/sh: 1: cannot open metatag.utils.DictMerger: No such file


## Display labelled sequences

And here are the results. The following table shows the taxonomic assignments of the query sequences which has been placed onto the nosZ tree and that passed the applied distance filters (as a quality control of the placement).

In [7]:
df = pd.read_csv(labeller.taxtable, sep="\t")
df.head()

,query_id,query_name,LWR,cluster_id,cluster_taxopath,taxopath
0,query_0,sp|P94127|NOSZ_ACHCY Nitrous-oxide reductase O...,0.5441,C0,Unspecified,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
1,query_1,sp|Q89XJ6|NOSZ_BRADU Nitrous-oxide reductase O...,0.5092,C0,Unspecified,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
2,query_10,sp|Q59746|NOSZ_RHIME Nitrous-oxide reductase O...,0.7280,C0,Unspecified,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...
3,query_11,sp|P19573|NOSZ_STUST Nitrous-oxide reductase O...,0.5244,C0,Unspecified,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...
4,query_2,sp|Q8YBC6|NOSZ_BRUME Nitrous-oxide reductase O...,0.8276,C0,Unspecified,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...


## Get citation

We can get the citation string by calling the `cite` method:

In [1]:
MetaTag.cite()

If you use this software, please cite it as below: 
Semidán Robaina Estévez (2022). MetaTag: Metagenome functional and taxonomical annotation through phylogenetic tree placement.(Version 0.1.0). Zenodo.
